# Extração dos dados da API do Olho vivo

Nesse notebook são extraídos os dados sobre os ônibus do município

Os dados são da API do Olho vivo: https://www.sptrans.com.br/desenvolvedores/api-do-olho-vivo-guia-de-referencia/

In [1]:
import pandas as pd
import time
import random
from requests import Session
from typing import Optional
from utils.save_csv import save_csv

In [2]:
class RequestMaker:

    def __init__(self, method:str, host:str, version:Optional[str]=None)->None:

        self.method = method
        self.host = host
        self.version = version
        self.session = Session()

        self.url_base = self.build_base_url()

    def build_base_url(self):

        url = f'{self.method}://{self.host}'

        if self.version:
            url = url+f'/v{self.version}'

        return url

    def build_params_str(self, params:dict)->str:

        params = [f"{key}={value}" for key, value in params.items()]
        params = '&'.join(params)
        params = f'?{params}'

        return params

    def build_request_url(self, endpoint:Optional[str]=None, params:Optional[dict]=None)->str:

        url = self.url_base
        if endpoint:
            url = self.url_base + '/' + endpoint.strip('/')
        
        if params:
            params_str = self.build_params_str(params)
            url = url + params_str
        
        return url
               
    def post(self,endpoint:Optional[str]=None, params:Optional[dict]=None, data:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo post em: {url}. Dados: {data}')
        with self.session.post(url, data=data) as r:
            if as_json:
                return r.json()
            return r.text
        
    def get(self, endpoint:Optional[str]=None, params:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo get em {url}.')
        with self.session.get(url) as r:
            if as_json:
                return r.json()
            return r.text



class APIOlhoVivoClient:

    HOST='api.olhovivo.sptrans.com.br'
    METHOD='http'
    VERSION='2.1'

    def __init__(self, token:str):

        self.request = RequestMaker(self.METHOD, self.HOST, self.VERSION)
        self.token=token

        self.autenticar()

    def autenticar(self)->None:

        endpoint = 'Login/Autenticar'
        params = {'token' : self.token}

        #a api nao retorna um json valido
        resp = self.request.post(endpoint, params, as_json=False)
        success = resp=='true'
        if not success:
            raise RuntimeError(f'Falha ao atenticar: {resp}')
        print('Autenticado com sucesso.')

    @property
    def posicao_atual_onibus(self):

        endpoint = 'Posicao'
        return self.request.get(endpoint)

In [3]:
token = '7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0'

In [4]:
api = APIOlhoVivoClient(token)

Fazendo post em: http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0. Dados: None
Autenticado com sucesso.


In [5]:
posicoes = []

espera = 30
for i in range(2):
    posicao_atual = api.posicao_atual_onibus
    posicoes.append(posicao_atual)
    time.sleep(espera)

posicoes

Fazendo get em http://api.olhovivo.sptrans.com.br/v2.1/Posicao.
Fazendo get em http://api.olhovivo.sptrans.com.br/v2.1/Posicao.


[{'hr': '12:47',
  'l': [{'c': '4014-10',
    'cl': 2187,
    'sl': 1,
    'lt0': 'TERM. VL. CARRÃO',
    'lt1': 'JD. VL. CARRÃO',
    'qv': 4,
    'vs': [{'p': 48729,
      'a': True,
      'ta': '2025-08-28T15:47:44Z',
      'py': -23.597995,
      'px': -46.471635,
      'sv': None,
      'is': None},
     {'p': 48062,
      'a': True,
      'ta': '2025-08-28T15:47:30Z',
      'py': -23.612996,
      'px': -46.456739999999996,
      'sv': None,
      'is': None},
     {'p': 48866,
      'a': True,
      'ta': '2025-08-28T15:47:26Z',
      'py': -23.5736935,
      'px': -46.50531049999999,
      'sv': None,
      'is': None},
     {'p': 48141,
      'a': True,
      'ta': '2025-08-28T15:47:18Z',
      'py': -23.6090855,
      'px': -46.4588275,
      'sv': None,
      'is': None}]},
   {'c': '648P-10',
    'cl': 32834,
    'sl': 2,
    'lt0': 'TERM. PINHEIROS',
    'lt1': 'TERM. CAPELINHA',
    'qv': 5,
    'vs': [{'p': 71429,
      'a': True,
      'ta': '2025-08-28T15:47:23Z',
    

In [6]:
rows = []

for l in posicoes[0]['l']:

    for veiculo in l['vs']:

        row = {
            'linha': l['c'],
            'id_onibus': veiculo['p'],
            'timestamp': veiculo['ta'],
            'lat': veiculo['py'],
            'lon': veiculo['px'] 
        }

        rows.append(row)

df_posicoes = pd.DataFrame(rows)

Aleatorização dos ônibus elétricos

In [7]:
df_posicoes['is_eletrico'] = [random.choice([True, False]) for i in range(len(df_posicoes))]

Aleatorização dos modelos de ônibus

In [8]:
consumo_diesel = pd.DataFrame({"tecnologia": ["Miniônibus", 
                                              "Midiônibus", 
                                              "Básico", 
                                              "Padron",
                                              "Articulado (18m)", 
                                              "Articulado (23m)", 
                                              "Biarticulado"],
    "sem_ar_l_km": [0.300, 0.400, 0.460, 0.550, 0.710, 0.750, 0.800],
    "sem_ar_kg_km": [0.252, 0.336, 0.386, 0.462, 0.596, 0.630, 0.672],
    "com_ar_l_km": [0.350, 0.470, 0.530, 0.630, 0.800, 0.850, 0.900],
    "com_ar_kg_km": [0.294, 0.395, 0.445, 0.529, 0.672, 0.714, 0.756]})

In [9]:
tipos = consumo_diesel["tecnologia"].tolist()

In [10]:
df_posicoes['modelo'] = [random.choice(tipos) for i in range(len(df_posicoes))]

Exibir e salvar

In [11]:
df_posicoes

,linha,id_onibus,timestamp,lat,lon,is_eletrico,modelo
0,4014-10,48729,2025-08-28T15:47:44Z,-23.597995,-46.471635,True,Padron
1,4014-10,48062,2025-08-28T15:47:30Z,-23.612996,-46.456740,False,Básico
2,4014-10,48866,2025-08-28T15:47:26Z,-23.573694,-46.505310,True,Miniônibus
3,4014-10,48141,2025-08-28T15:47:18Z,-23.609085,-46.458827,False,Articulado (18m)
4,648P-10,71429,2025-08-28T15:47:23Z,-23.655038,-46.761806,False,Padron
...,...,...,...,...,...,...,...
9940,6051-31,66621,2025-08-28T15:47:52Z,-23.762822,-46.688934,False,Articulado (23m)
9941,6801-10,72831,2025-08-28T15:47:55Z,-23.660584,-46.733610,True,Básico
9942,2767-10,31743,2025-08-28T15:47:44Z,-23.510844,-46.508745,True,Padron
9943,4008-21,48671,2025-08-28T15:47:57Z,-23.569003,-46.439658,False,Midiônibus


In [12]:
save_csv(df_posicoes, 'df_posicoes.csv')

Base salva em data\df_posicoes.csv
